In [1]:
import features_mp as ft
t_data = ft.gen_n_feature(50000)

S:\anaconda\envs\tf\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


read the data for pos_vec_train_0_50000
data for pos_vec_train_0_50000 read


In [2]:
import pandas as pd
import numpy as np
from time import time

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
import sklearn

from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
%matplotlib inline
import matplotlib.pyplot as pl
import matplotlib.patches as mpatches
import numpy as np
from IPython import get_ipython

rand_number = 42



In [3]:
def one_hot(y):
    oc = OneHotEncoder(categories='auto')
    y_r = y.reshape(-1,1)
    oc.fit(y_r)
    r = oc.transform(y_r).toarray()
    return r


def t_model(gbm):
    y_trainp = gbm.predict_proba(X_train)
    y_testp  = gbm.predict_proba(X_test)
    print(log_loss(y_train, y_trainp), log_loss(y_test, y_testp))


def tt_model(gbm):
    gbm = gbm.fit(X_train, y_train)
    t_model(gbm)

out_features = ['question1', 'question2', 'is_duplicate']
features = [x for x in t_data.columns if x not in out_features]

t_data = t_data[t_data!=np.inf].dropna()
feature_data = t_data.drop_duplicates(features, keep='last')
input_data = feature_data[features]
input_data = input_data.astype(np.float64)
result = feature_data[['is_duplicate']]
result = result.values.ravel()

def time_cnt(f, tag="func"):
    t_start = time()
    ret = f()
    t_end = time()
    t_used = t_end - t_start
    print("function '%s' use: %f s" % (tag, t_used))
    
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(input_data, result, test_size = 0.2, random_state = 0,
                                                    stratify = result)

In [9]:
def search_model(model, params):
    gs = GridSearchCV(estimator=model, param_grid=params, 
                  scoring="neg_log_loss", cv=5, n_jobs=6)
    gs.fit(X_train, y_train)
    print(gs.best_params_, gs.best_score_)
    t_model(gs.best_estimator_)

In [4]:
# sorted(sklearn.metrics.SCORERS.keys())
X_train.shape, X_test.shape

((39808, 212), (9952, 212))

In [9]:
# base model
model = LGBMClassifier(n_jobs=4, num_thread=4)
tt_model(model)

0.36419910325099414 0.4385182119916422


In [10]:
from sklearn.model_selection import GridSearchCV
params={
    'max_depth': [15, 20, 25, 30, 35],
    'num_leaves': [20, 35, 50]
}

gbm = LGBMClassifier(n_jobs=8)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss', cv=3, verbose=1, n_jobs=3)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  45 out of  45 | elapsed:  3.5min finished


{'max_depth': 25, 'num_leaves': 50} -0.43267117145260253


In [11]:
best = LGBMClassifier(num_thread=4, n_jobs=4, max_depth=25, num_leaves=50)
tt_model(best)

0.32112639483570904 0.43299972735853454


In [12]:
params={
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'bagging_freq': [20, 35, 50, 70]
}
gbm = LGBMClassifier(n_jobs=4, max_depth=20, num_leaves=50)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss', cv=3, verbose=1, n_jobs=3)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)
t_model(gsearch1.best_estimator_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  2.4min
[Parallel(n_jobs=3)]: Done  60 out of  60 | elapsed:  3.2min finished


{'bagging_freq': 20, 'learning_rate': 0.1} -0.4338827864115586
0.32064693628896285 0.4338503448670902


In [15]:
params={
    'bagging_freq': [20, 19, 18]
}
gbm = LGBMClassifier(n_jobs=4, max_depth=20, num_leaves=50, learning_rate=0.1)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss', cv=3, verbose=1, n_jobs=3)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)
t_model(gsearch1.best_estimator_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   9 out of   9 | elapsed:   24.4s finished


{'bagging_freq': 20} -0.4338827864115586
0.32064693628896285 0.4338503448670902


In [16]:
params={
    'lambda_l1': [0, 0.1, 0.4, 0.5, 0.6],
    'lambda_l2': [0, 10, 15, 35, 40],
}
gbm = LGBMClassifier(n_jobs=8, max_depth=20, num_leaves=50, learning_rate=0.1)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss', cv=3, verbose=1, n_jobs=3)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)
t_model(gsearch1.best_estimator_)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  1.9min
[Parallel(n_jobs=3)]: Done  75 out of  75 | elapsed:  3.1min finished


{'lambda_l1': 0, 'lambda_l2': 15} -0.4304202684501281
0.3355672871743284 0.4293634599592099


In [17]:
params={
    'lambda_l1': [0, 0.03, 0.05, 0.07, 0.09],
    'lambda_l2': [10,13,15,18,20],
}
gbm = LGBMClassifier(n_jobs=8, max_depth=20, num_leaves=50, learning_rate=0.1)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss',
                        cv=3, verbose=1, n_jobs=4)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)
t_model(gsearch1.best_estimator_)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done  75 out of  75 | elapsed:  2.9min finished


{'lambda_l1': 0, 'lambda_l2': 15} -0.4304202684501281
0.3355672871743284 0.4293634599592099


In [18]:
params={
    'feature_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
    'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
#     'cat_smooth': [1, 10, 15, 20, 35]
}
gbm = LGBMClassifier(n_jobs=4, max_depth=20,num_leaves=50,
                     learning_rate=0.1, lambda_l1=0,lambda_l2=15)

gsearch1 = GridSearchCV(gbm, param_grid=params,
                        scoring='neg_log_loss', cv=3, verbose=1, n_jobs=4)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)
t_model(gsearch1.best_estimator_)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done  75 out of  75 | elapsed:  2.5min finished


{'bagging_fraction': 0.6, 'feature_fraction': 0.7} -0.4307992929075793
0.33898751509132263 0.4319067963674971


In [20]:
params={
    'cat_smooth': [0.1, 0.3, 0.7, 1, 3, 5, 8]
}
gbm = LGBMClassifier(n_jobs=3, max_depth=20,num_leaves=50,
                     learning_rate=0.1, lambda_l1=0,lambda_l2=15)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss',
                        cv=3, verbose=1, n_jobs=4)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)
t_model(gsearch1.best_estimator_)

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  21 out of  21 | elapsed:   46.2s finished


{'cat_smooth': 0.1} -0.4304202684501281
0.3355672871743284 0.4293634599592099


## Tuned LightGBM

In [21]:
gbm = LGBMClassifier(n_jobs=3, max_depth=20,num_leaves=50,
                     learning_rate=0.1, lambda_l1=0,lambda_l2=15)
tt_model(gbm)

0.3355672871743284 0.4293634599592099


In [22]:
param_test_last1 = {"penalty": ["l1"], "solver": ['liblinear']}
rl_clf = LogisticRegression()
gs = GridSearchCV(estimator=rl_clf, param_grid=param_test_last1,
                  scoring='neg_log_loss', cv=5, n_jobs=8)
gs.fit(X_train, y_train)
print(gs.best_params_, gs.best_score_)
print(t_model(gs.best_estimator_))

{'penalty': 'l1', 'solver': 'liblinear'} -0.4773198218777107
0.4726993214019336 0.4860577805289532
None


In [6]:
param_test_last2 = {"penalty": ["l2"], "solver": ['newton-cg', 'lbfgs', 'liblinear', 'sag']}
rl_clf = LogisticRegression()
gs = GridSearchCV(estimator=rl_clf, param_grid=param_test_last2,
                  scoring='neg_log_loss', cv=5, n_jobs=8)
gs.fit(X_train, y_train)
print(gs.best_params_, gs.best_score_)
print(t_model(gs.best_estimator_))

S:\anaconda\envs\tf\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


{'penalty': 'l2', 'solver': 'newton-cg'} -0.47885358964912916
0.47453683930331875 0.48706166706446935
None


In [10]:
rl_clf = LogisticRegression(penalty='l1', solver='liblinear')
search_model(rl_clf, {"C": [0.01,0.1,1,5,10,15]})

S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


{'C': 15} -0.47418254498830215
0.4682089361068081 0.48179063270267747


In [11]:
rl_clf = LogisticRegression(penalty='l1', solver='liblinear')
search_model(rl_clf, {"C": [13, 15, 20, 30, 50]})

{'C': 30} -0.47402783285131106
0.4679132862017766 0.4810722664705136


In [ ]:
rl_clf = LogisticRegression(penalty='l1', solver='liblinear')
search_model(rl_clf, {"C": [25 30, 35, 40, 45]})

## Tuned RL

In [12]:
rl_clf = LogisticRegression(penalty='l1', solver='liblinear', C=30)
tt_model(rl_clf)

0.46794262367489087 0.4811051673834153


In [17]:
sgd_clf = SGDClassifier(n_jobs=5, loss='log',random_state=rand_number)
tt_model(sgd_clf)
sgd_clf

S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


11.854339969988084 11.92880524169167


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=5, penalty='l2',
       power_t=0.5, random_state=42, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [24]:
sgd_clf = SGDClassifier(n_jobs=5, random_state=rand_number)
search_model(sgd_clf, {"loss": ["log", "modified_huber"]})

S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


{'loss': 'log'} -11.982811848366763


S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


11.854339969988084 11.92880524169167


In [25]:
sgd_clf = SGDClassifier(loss="log", n_jobs=3, random_state=rand_number)
search_model(sgd_clf, {"penalty": ['l2', 'l1', 'elasticnet']})

S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


{'penalty': 'l1'} -11.849631915688299


S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


10.337422363458993 10.623319121604244


In [26]:
sgd_clf = SGDClassifier(penalty='l1', loss="log", n_jobs=3,
                        random_state=rand_number)
search_model(sgd_clf, {"max_iter": [50, 100, 200, 300, 400, 500], 
                       "tol": [0.0001, 0.001, 0.01, 0.1, 0.00001]})

{'max_iter': 300, 'tol': 0.0001} -9.930484967268876


S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


8.895253021219034 9.134993716067525


## Tuned SGD

In [28]:
sgd_clf = SGDClassifier(penalty='l1', loss="log", max_iter=300, tol=0.0001,
                        n_jobs=3, random_state=rand_number)
tt_model(sgd_clf)

S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
S:\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


8.895253021219034 9.134993716067525
